In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
data_path = 'C:\\Users\\zer0nu11\\Documents\\grad\\skolkovohack2022\\data\\for_hack_2022\\'
test_path = 'C:\\Users\\zer0nu11\\Documents\\grad\\skolkovohack2022\\data\\for_hack_2022\\test\\'

In [5]:
jobs_labels = ['JobId','Status','Name','Region','Description']
test_jobs_labels = ['JobId','Status','Name','Region','Description','nan1','nan2','nan3']
candidates_labels = ['CandidateId', 'Position', 'Sex', 'Citizenship', 'Age', 'Salary',
       'Langs', 'DriverLicense', 'Subway', 'Skills', 'Employment', 'Schedule',
       'CandidateRegion','DateCreated','JobId','CandidateStatusId','Status']
candidates_education_labels = ['CandidateId', 'University', 'Faculty', 'GraduateYear']
candidates_workplaces_labels = ['CandidateId', 'Position', 'FromYear', 'FromMonth', 'ToYear',
       'ToMonth']

data_jobs = pd.read_csv(data_path+'data_jobs.csv',sep=';',names=jobs_labels)
data_candidates_workplaces = pd.read_csv(data_path+'data_candidates_work_places.csv',sep=';',names=candidates_workplaces_labels)
data_candidates_education = pd.read_csv(data_path+'data_candidates_education.csv',sep=';',names=candidates_education_labels)
data_candidates = pd.read_csv(data_path+'data_candidates.csv',sep=';',names=candidates_labels)

test_jobs = pd.read_csv(test_path+'test_jobs.csv',sep=';',names=test_jobs_labels)
test_candidates_workplaces = pd.read_csv(test_path+'test_candidates_workplaces.csv',sep=';')
test_candidates_education = pd.read_csv(test_path+'test_candidates_education.csv',sep=';')
test_candidates = pd.read_csv(test_path+'test_candidates.csv',sep=';')

In [ ]:
# assert test_jobs.shape[1] == data_jobs.shape[1] # test_jobs.shape БОЛЬШЕ НА 3 ФИЧИ!
# assert test_candidates.shape[1] == data_candidates.shape[1] # test_candidates.shape МЕНЬШЕ НА 4 ФИЧИ!
assert test_candidates_education.shape[1] == data_candidates_education.shape[1]
assert test_candidates_workplaces.shape[1] == data_candidates_workplaces.shape[1]

In [6]:
test_jobs

,JobId,Status,Name,Region,Description,nan1,nan2,nan3
0,140952,2,Водитель автомобиля большегруза с прицепом,Санкт-Петербург,<p><strong>ОБЯЗАННОСТИ:</strong><span></span><...,2197,1,1
1,76137,2,водитель на своем авто,Москва,"<p>Начался сезон отпусков, по этой причине отк...",2869,1,1
2,119729,4,Водитель,Москва,"<p>Наш клиент, крупный холдинг рассматривает к...",2869,1,1
3,62241,2,Водитель на своем авто,Москва,Стаж вождения от 3-х лет;<br />Образование не ...,2869,1,1
4,63795,2,Водитель,Санкт-Петербург,<p>Обязанности:<br /><br />Качественная достав...,2869,1,1
5,94661,2,водитель на личном авто,Москва,ищем курьера выходного дня на своем авто,2869,1,1
6,120695,2,Водитель-экспедитор 1/ Водитель,Москва,<p><strong>Обязанности:</strong></p> <ul> <li>...,2869,1,1
7,157891,2,водитель курьер на авто компании,Москва,"<p>водитель на авто компании 6% от заказов, во...",2869,1,1
8,165605,2,Менеджер по продажам,Санкт-Петербург,Обязанности:Работа с уже имеющейся клиентской ...,2869,1,1
9,241719,2,Водитель-экспедитор,Москва,<p><strong>ЗАДАЧИ:</strong></p> <ul> <li>Доста...,2869,1,1
